# scales

> Numeric and named scale builders for Tailwind CSS utilities

In [ ]:
#| default_exp builders.scales

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, List, Union, Optional, Tuple, Callable, Any
from dataclasses import dataclass
from cjm_fasthtml_tailwind.core.base import (
    NamedScale, CONTAINER_SCALES, TailwindScale, 
    BaseUtility, StandardUtility, DirectionalUtility,
    UtilityFactory, is_numeric_scale, BaseFactory, SingleValueUtility
)

## Numeric Scale Definitions

Define the standard numeric scales used by Tailwind CSS:

In [ ]:
#| export
NUMERIC_SCALE = list(range(97)) # Standard spacing scale (0-96)

DECIMAL_SCALE = [0.5, 1.5, 2.5, 3.5] # Common decimal scales

SPACING_SCALE = sorted(set(NUMERIC_SCALE + DECIMAL_SCALE)) # Extended spacing scale with decimals

FRACTION_DENOMINATORS = [2, 3, 4, 5, 6, 12] # Fraction denominators supported by Tailwind

In [ ]:
#| export
def generate_fractions(
) -> List[str]:  # List of all valid Tailwind fraction strings sorted by value
    """Generate all valid Tailwind fractions."""
    fractions = []
    for denominator in FRACTION_DENOMINATORS:
        for numerator in range(1, denominator):
            fractions.append(f"{numerator}/{denominator}")
    return sorted(set(fractions), key=lambda x: eval(x))  # Sort by actual value

FRACTIONS = generate_fractions() # Pre-generate fractions

## Scale Builders

Builders for creating utilities with different scale types:

In [ ]:
#| export
@dataclass
class ScaleConfig:
    """Configuration for a scale builder."""
    numeric: bool = True  # Support numeric scales (0-96)
    decimals: bool = False  # Support decimal scales (0.5, 1.5, etc.)
    fractions: bool = False  # Support fractions (1/2, 1/3, etc.)
    named: Optional[List[NamedScale]] = None  # Named scales (xs, sm, md, etc.)
    special: Optional[Dict[str, str]] = None  # Special values (auto, full, screen, etc.)
    negative: bool = False  # Support negative values

In [ ]:
#| export
class ScaledUtility(StandardUtility):
    """Utility class with scale support."""
    
    def __init__(
        self, 
        prefix: str,  # The utility prefix (e.g., 'w', 'h', 'p')
        config: ScaleConfig,  # Configuration defining valid scales and values
        negative: bool = False  # Whether this is a negative variant
    ):
        """Initialize with prefix and scale configuration."""
        self.config = config
        self.negative = negative
        full_prefix = f"-{prefix}" if negative else prefix
        super().__init__(full_prefix)
    
    def _format_value(
        self,
        value: TailwindScale  # The value to format (can be numeric, fraction, or string)
    ) -> str:  # The formatted value string with appropriate wrapping
        """
        Format value according to Tailwind conventions with scale awareness:
        - Values with spaces: wrapped in brackets (arbitrary)
        - String numbers not in scale: wrapped in brackets
        - Otherwise: use standard formatting
        """
        # First check if it's a string containing spaces
        if isinstance(value, str) and ' ' in value:
            return f"[{value}]"
        
        # Check if it's a string number that should be arbitrary
        if isinstance(value, str) and value.isdigit():
            # Check if this number is in the valid numeric scale for this utility
            if self.config.numeric and int(value) in NUMERIC_SCALE:
                # It's a valid numeric scale value, format normally
                return value
            else:
                # It's outside the scale, treat as arbitrary
                return f"[{value}]"
        
        # Otherwise use the standard formatting from parent class
        return super()._format_value(value)
    
    def get_valid_values(
        self
    ) -> List[Union[str, int, float]]:  # List of all valid values for this utility
        """Get all valid values for this utility."""
        values = []
        
        # Add numeric scales
        if self.config.numeric:
            values.extend(NUMERIC_SCALE)
        
        # Add decimal scales
        if self.config.decimals:
            values.extend(DECIMAL_SCALE)
        
        # Add fractions
        if self.config.fractions:
            values.extend(FRACTIONS)
        
        # Add named scales
        if self.config.named:
            values.extend([scale.name for scale in self.config.named])
        
        # Add special values
        if self.config.special:
            values.extend(self.config.special.keys())
        
        return values

## Scale Factory

Enhanced factory that supports all scale types:

In [ ]:
#| export
class ScaledFactory(UtilityFactory[ScaledUtility]):
    """Factory for creating scaled utilities with enhanced attribute access."""
    
    def __init__(
        self, 
        prefix: str,  # The utility prefix (e.g., 'w', 'h', 'p')
        config: ScaleConfig,  # Configuration defining valid scales and values
        doc: Optional[str] = None  # Optional documentation string
    ):
        """Initialize with prefix and scale configuration."""
        self.config = config
        super().__init__(ScaledUtility, prefix, doc)
    
    def __call__(
        self, 
        value: Optional[TailwindScale] = None,  # The value to apply to the utility
        negative: bool = False  # Whether to create a negative variant
    ) -> ScaledUtility:  # A new scaled utility instance
        """Create a utility instance with optional value."""
        instance = ScaledUtility(self.prefix, self.config, negative)
        if value is not None:
            instance._value = instance._format_value(value)
        return instance
    
    def __getattr__(
        self,
        name: str  # The attribute name to convert to a utility value
    ) -> ScaledUtility:  # A new scaled utility instance with the attribute as value
        """
        Handle attribute access for named values.
        Examples: w.full, h.screen, p.auto, w._2xl
        """
        # Handle negative prefix
        if name.startswith("neg_"):
            actual_name = name[4:]  # Remove "neg_" prefix
            # Handle leading underscore for negative values too
            if actual_name.startswith("_") and len(actual_name) > 1:
                actual_name = actual_name[1:]
            instance = ScaledUtility(self.prefix, self.config, negative=True)
            instance._value = actual_name.replace("_", "-")
            return instance
        
        # Handle leading underscore (for values that start with numbers)
        if name.startswith("_") and len(name) > 1:
            # Remove the leading underscore
            actual_name = name[1:]
        else:
            actual_name = name
        
        # Regular attribute access
        instance = ScaledUtility(self.prefix, self.config)
        instance._value = actual_name.replace("_", "-")
        return instance
    
    @property
    def negative(
        self
    ) -> 'NegativeFactory':  # A factory for creating negative variants
        """Return a negative variant factory."""
        return NegativeFactory(self.prefix, self.config)
    
    def get_info(
        self
    ) -> Dict[str, Any]:  # Dictionary with factory information
        """Get detailed information about this scaled factory."""
        valid_inputs = []
        
        if self.config.numeric:
            valid_inputs.append(f"Numeric scales: 0-{max(NUMERIC_SCALE)}")
        
        if self.config.decimals:
            valid_inputs.append(f"Decimal scales: {', '.join(str(d) for d in DECIMAL_SCALE)}")
        
        if self.config.fractions:
            valid_inputs.append(f"Fractions: {len(FRACTIONS)} supported (e.g., 1/2, 2/3, 3/4)")
        
        if self.config.named:
            named_values = [scale.name for scale in self.config.named]
            valid_inputs.append(f"Named scales: {', '.join(named_values)}")
        
        if self.config.special:
            valid_inputs.append(f"Special values: {', '.join(self.config.special.keys())}")
        
        valid_inputs.append("Arbitrary values: Any string with CSS units (e.g., '10px', '2.5rem')")
        valid_inputs.append("Custom properties: CSS variables starting with -- (e.g., '--spacing')")
        
        options = {
            'prefix': self.prefix,
            'supports_negative': self.config.negative
        }
        
        if self.config.negative:
            options['negative_access'] = 'Use .negative property or negative=True parameter'
        
        return {
            'description': self._doc,
            'valid_inputs': valid_inputs,
            'options': options
        }

In [ ]:
#| export
class NegativeFactory:
    """Factory for creating negative variants."""
    
    def __init__(
        self,
        prefix: str,  # The utility prefix (e.g., 'm', 'inset')
        config: ScaleConfig  # Configuration defining valid scales and values
    ):
        "Initialize with prefix and scale configuration."
        self.prefix = prefix
        self.config = config
    
    def __call__(
        self,
        value: TailwindScale  # The value to apply to the negative utility
    ) -> ScaledUtility:  # A new negative scaled utility instance
        """Create a negative utility instance."""
        instance = ScaledUtility(self.prefix, self.config, negative=True)
        instance._value = instance._format_value(value)
        return instance
    
    def __getattr__(
        self,
        name: str  # The attribute name to convert to a negative utility value
    ) -> ScaledUtility:  # A new negative scaled utility instance
        """Handle attribute access for negative named values."""
        instance = ScaledUtility(self.prefix, self.config, negative=True)
        instance._value = name.replace("_", "-")
        return instance

## Directional Scale Factory

Factory for utilities with directional variants:

In [ ]:
#| export
class DirectionalScaledUtility(DirectionalUtility):
    """Directional utility with scale support."""
    
    def __init__(
        self, 
        prefix: str,  # The base utility prefix (e.g., 'p' for padding)
        direction: Optional[str],  # The direction suffix ('t', 'r', 'b', 'l', 'x', 'y')
        config: ScaleConfig,  # Configuration defining valid scales and values
        negative: bool = False  # Whether this is a negative variant
    ):
        """Initialize with prefix, direction, and scale configuration."""
        self.config = config
        self.negative = negative
        # Apply negative prefix before direction
        base_prefix = f"-{prefix}" if negative else prefix
        super().__init__(base_prefix, direction)

In [ ]:
#| export
class DirectionalScaledFactory(BaseFactory):
    """Factory for creating directional scaled utilities."""
    
    def __init__(
        self, 
        prefix: str,  # The base utility prefix (e.g., 'p' for padding, 'm' for margin)
        config: ScaleConfig,  # Configuration defining valid scales and values
        doc: Optional[str] = None  # Optional documentation string
    ):
        """Initialize with prefix and scale configuration."""
        doc = doc or f"Factory for {prefix} utilities with directional variants"
        super().__init__(doc)
        self.prefix = prefix
        self.config = config
        
        # Create direction-specific factories
        self.t = ScaledFactory(f"{prefix}t", config, f"Top {prefix} utilities")  # top
        self.r = ScaledFactory(f"{prefix}r", config, f"Right {prefix} utilities")  # right
        self.b = ScaledFactory(f"{prefix}b", config, f"Bottom {prefix} utilities")  # bottom
        self.l = ScaledFactory(f"{prefix}l", config, f"Left {prefix} utilities")  # left
        self.x = ScaledFactory(f"{prefix}x", config, f"Horizontal {prefix} utilities")  # horizontal
        self.y = ScaledFactory(f"{prefix}y", config, f"Vertical {prefix} utilities")  # vertical
    
    def __call__(
        self, 
        value: Optional[TailwindScale] = None,  # The value to apply to all directions
        negative: bool = False  # Whether to create a negative variant
    ) -> ScaledUtility:  # A new scaled utility for all directions
        """Create a utility instance for all directions."""
        return ScaledFactory(self.prefix, self.config, f"All sides {self.prefix} utilities")(value, negative)
    
    def __getattr__(
        self,
        name: str  # The attribute name to convert to a utility value
    ) -> ScaledUtility:  # A new scaled utility instance
        """Handle attribute access for named values."""
        return ScaledFactory(self.prefix, self.config).__getattr__(name)
    
    @property
    def negative(
        self
    ) -> 'NegativeFactory':  # A factory for creating negative variants
        """Return a negative variant factory."""
        return NegativeFactory(self.prefix, self.config)
    
    def get_info(
        self
    ) -> Dict[str, Any]:  # Dictionary with factory information
        """Get detailed information about this directional factory."""
        # Get valid inputs from the config (same as ScaledFactory)
        valid_inputs = []
        
        if self.config.numeric:
            valid_inputs.append(f"Numeric scales: 0-{max(NUMERIC_SCALE)}")
        
        if self.config.decimals:
            valid_inputs.append(f"Decimal scales: {', '.join(str(d) for d in DECIMAL_SCALE)}")
        
        if self.config.fractions:
            valid_inputs.append(f"Fractions: {len(FRACTIONS)} supported (e.g., 1/2, 2/3, 3/4)")
        
        if self.config.named:
            named_values = [scale.name for scale in self.config.named]
            valid_inputs.append(f"Named scales: {', '.join(named_values)}")
        
        if self.config.special:
            valid_inputs.append(f"Special values: {', '.join(self.config.special.keys())}")
        
        valid_inputs.append("Arbitrary values: Any string with CSS units (e.g., '10px', '2.5rem')")
        valid_inputs.append("Custom properties: CSS variables starting with -- (e.g., '--spacing')")
        
        options = {
            'prefix': self.prefix,
            'supports_negative': self.config.negative,
            'directional_variants': {
                't': 'top',
                'r': 'right', 
                'b': 'bottom',
                'l': 'left',
                'x': 'horizontal (left and right)',
                'y': 'vertical (top and bottom)'
            }
        }
        
        if self.config.negative:
            options['negative_access'] = 'Use .negative property or negative=True parameter'
        
        return {
            'description': self._doc,
            'valid_inputs': valid_inputs,
            'options': options
        }

## Pre-configured Scale Configs

Common scale configurations for different utility types:

In [ ]:
#| export
SPACING_CONFIG = ScaleConfig( # Spacing configuration (padding, margin, gap)
    numeric=True,
    decimals=True,
    fractions=False,
    named=None,
    special={"px": "px", "auto": "auto"},
    negative=True
)

SIZE_CONFIG = ScaleConfig( # Size configuration (width, height)
    numeric=True,
    decimals=True,
    fractions=True,
    named=CONTAINER_SCALES,
    special={
        "auto": "auto",
        "px": "px", 
        "full": "full",
        "screen": "screen",
        "svw": "svw",
        "svh": "svh",
        "lvw": "lvw", 
        "lvh": "lvh",
        "dvw": "dvw",
        "dvh": "dvh",
        "min": "min",
        "max": "max",
        "fit": "fit",
        "lh": "lh"  # line-height unit for height
    },
    negative=False
)

INSET_CONFIG = ScaleConfig( # Inset configuration (top, right, bottom, left)
    numeric=True,
    decimals=True,
    fractions=True,
    named=None,
    special={
        "auto": "auto",
        "px": "px",
        "full": "full"
    },
    negative=True
)

## Examples

Test the scale builders with various configurations:

In [ ]:
# Test basic numeric scales
w = ScaledFactory("w", SIZE_CONFIG)
assert str(w(4)) == "w-4"
assert str(w(2.5)) == "w-2.5"
assert str(w(0)) == "w-0"

In [ ]:
# Test fraction support
assert str(w("1/2")) == "w-1/2"
assert str(w("3/4")) == "w-3/4"
assert str(w("2/3")) == "w-2/3"

In [ ]:
# Test named scales
assert str(w.xs) == "w-xs"
assert str(w.sm) == "w-sm"
assert str(w.lg) == "w-lg"
assert str(w._2xl) == "w-2xl"  # Python identifiers can't start with numbers

In [ ]:
# Test special values
assert str(w.auto) == "w-auto"
assert str(w.full) == "w-full"
assert str(w.screen) == "w-screen"
assert str(w.px) == "w-px"

In [ ]:
# Test arbitrary values
assert str(w("10px")) == "w-[10px]"
assert str(w("2.5rem")) == "w-[2.5rem]"
assert str(w("calc(100% - 20px)")) == "w-[calc(100% - 20px)]"

In [ ]:
# Test custom properties
assert str(w("--custom-width")) == "w-(--custom-width)"

### Test Directional Scales

In [ ]:
# Test directional factory
p = DirectionalScaledFactory("p", SPACING_CONFIG)

# Test all directions
assert str(p(4)) == "p-4"
assert str(p.t(4)) == "pt-4"
assert str(p.r(4)) == "pr-4"
assert str(p.b(4)) == "pb-4"
assert str(p.l(4)) == "pl-4"
assert str(p.x(4)) == "px-4"
assert str(p.y(4)) == "py-4"

In [ ]:
# Test directional with special values
assert str(p.x.auto) == "px-auto"
assert str(p.y(0)) == "py-0"

### Test Negative Values

In [ ]:
# Test margin with negative values
m = DirectionalScaledFactory("m", SPACING_CONFIG)

# Test negative numeric values
assert str(m(4, negative=True)) == "-m-4"
assert str(m.negative(4)) == "-m-4"
assert str(m.t.negative(2)) == "-mt-2"
assert str(m.x.negative(8)) == "-mx-8"

In [ ]:
# Test negative special values
assert str(m.negative.px) == "-m-px"
assert str(m.x.negative.px) == "-mx-px"

### Test Inset with Negative Values

In [ ]:
# Test inset (top/right/bottom/left)
inset = DirectionalScaledFactory("inset", INSET_CONFIG)

# Regular values
assert str(inset(4)) == "inset-4"
assert str(inset("1/2")) == "inset-1/2"
assert str(inset.auto) == "inset-auto"
assert str(inset.full) == "inset-full"

# Negative values
assert str(inset.negative(4)) == "-inset-4"
assert str(inset.negative("1/2")) == "-inset-1/2"
assert str(inset.negative.full) == "-inset-full"

### Test All Fractions

In [ ]:
# Show all generated fractions
print("Generated fractions:")
print(FRACTIONS)

Generated fractions:
['1/12', '2/12', '1/6', '1/5', '1/4', '3/12', '4/12', '1/3', '2/6', '2/5', '5/12', '3/6', '6/12', '2/4', '1/2', '7/12', '3/5', '4/6', '2/3', '8/12', '3/4', '9/12', '4/5', '5/6', '10/12', '11/12']


In [ ]:
# Test fraction edge cases
h = ScaledFactory("h", SIZE_CONFIG)
assert str(h("1/2")) == "h-1/2"
assert str(h("1/3")) == "h-1/3"
assert str(h("2/3")) == "h-2/3"
assert str(h("1/4")) == "h-1/4"
assert str(h("3/4")) == "h-3/4"
assert str(h("1/6")) == "h-1/6"
assert str(h("5/6")) == "h-5/6"

### Test Class Combination

In [ ]:
# Test combining multiple utilities
from cjm_fasthtml_tailwind.core.base import combine_classes
w = ScaledFactory("w", SIZE_CONFIG)
h = ScaledFactory("h", SIZE_CONFIG)
p = DirectionalScaledFactory("p", SPACING_CONFIG)
m = DirectionalScaledFactory("m", SPACING_CONFIG)

# Create various utilities
w_util = w(32)
h_util = h.full
p_util = p.x(4)
m_util = m.y.auto

# Combine them
classes = combine_classes(w_util, h_util, p_util, m_util, "flex", "items-center")
assert classes == "w-32 h-full px-4 my-auto flex items-center"

## Helper Functions

Utility functions for working with scales:

In [ ]:
#| export
def list_scale_values(
    config: ScaleConfig  # The scale configuration to extract values from
) -> Dict[str, List[Union[str, int, float]]]:  # Dictionary mapping scale types to their values
    """List all possible values for a scale configuration."""
    values = {
        "numeric": [],
        "decimals": [],
        "fractions": [],
        "named": [],
        "special": []
    }
    
    if config.numeric:
        values["numeric"] = NUMERIC_SCALE
    
    if config.decimals:
        values["decimals"] = DECIMAL_SCALE
    
    if config.fractions:
        values["fractions"] = FRACTIONS
    
    if config.named:
        values["named"] = [scale.name for scale in config.named]
    
    if config.special:
        values["special"] = list(config.special.keys())
    
    return values

In [ ]:
# Test listing scale values
size_values = list_scale_values(SIZE_CONFIG)
print(f"Size scale has {len(size_values['numeric'])} numeric values")
print(f"Size scale has {len(size_values['fractions'])} fraction values")
print(f"Named sizes: {size_values['named'][:5]}...")  # First 5
print(f"Special sizes: {size_values['special']}")

Size scale has 97 numeric values
Size scale has 26 fraction values
Named sizes: ['3xs', '2xs', 'xs', 'sm', 'md']...
Special sizes: ['auto', 'px', 'full', 'screen', 'svw', 'svh', 'lvw', 'lvh', 'dvw', 'dvh', 'min', 'max', 'fit', 'lh']


## Simple Factory

A factory for utilities that use simple string values with dot notation access:

In [ ]:
#| export
class SimpleFactory(BaseFactory):
    """Factory for utilities that are simple string values with modifier support."""
    
    def __init__(
        self,
        values_dict: Dict[str, str],  # Dictionary mapping attribute names to CSS values
        doc: Optional[str] = None  # Optional documentation string
    ):
        "Initialize with a dictionary of values."
        doc = doc or "Factory for simple utility values"
        super().__init__(doc)
        self._values = values_dict
        # Cache utility instances for each value
        self._utility_cache = {}
    
    def __getattr__(
        self,
        name: str  # The attribute name to look up
    ) -> Union[SingleValueUtility, Any]:  # A SingleValueUtility instance or attribute
        "Get utility instance by attribute name, converting underscores to hyphens."
        # Handle underscore to hyphen conversion for multi-word values
        key = name.replace("_", "-")
        css_value = None
        
        if key in self._values:
            css_value = self._values[key]
        elif name in self._values:
            css_value = self._values[name]
        
        if css_value is not None:
            # Return cached utility instance or create new one
            if css_value not in self._utility_cache:
                from cjm_fasthtml_tailwind.core.base import SingleValueUtility
                self._utility_cache[css_value] = SingleValueUtility(css_value)
            return self._utility_cache[css_value]
        
        raise AttributeError(f"'{self.__class__.__name__}' object has no attribute '{name}'")
    
    def get_info(
        self
    ) -> Dict[str, Any]:  # Dictionary with factory information
        """Get information about this simple factory."""
        return {
            'description': self._doc,
            'valid_inputs': 'No inputs - access values as attributes with modifier support',
            'options': {
                'available_values': list(self._values.keys()),
                'supports_modifiers': True
            }
        }

In [ ]:
# Test SimpleFactory with modifier support
test_values = {
    "auto": "auto-value",
    "none": "none-value",
    "multi-word": "multi-word-value"
}

factory = SimpleFactory(test_values)

# Test basic access (now returns utilities)
assert str(factory.auto) == "auto-value"
assert str(factory.none) == "none-value"
assert str(factory.multi_word) == "multi-word-value"

# Test with modifiers
assert str(factory.auto.hover) == "hover:auto-value"
assert str(factory.none.md) == "md:none-value"
assert str(factory.multi_word.dark) == "dark:multi-word-value"

# Test chained modifiers
assert str(factory.auto.hover.lg) == "lg:hover:auto-value"
assert str(factory.none.dark.focus) == "focus:dark:none-value"

# Test that cached instances work correctly
auto1 = factory.auto
auto2 = factory.auto
assert auto1 is auto2  # Should be the same cached instance

print("✅ SimpleFactory with modifiers tests passed!")

✅ SimpleFactory with modifiers tests passed!


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()